In [6]:
!apt-get -qq install -y fonts-freefont-ttf > /dev/null

import os, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from google.colab import files

# ---- Style (Journal) ----
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['FreeSerif']
mpl.rcParams['mathtext.fontset'] = 'dejavuserif'
mpl.rcParams['axes.unicode_minus'] = False
mpl.rcParams['font.size'] = 12
mpl.rcParams['axes.titlesize'] = 14
mpl.rcParams['axes.labelsize'] = 12
mpl.rcParams['legend.fontsize'] = 11

DPI = 600

# ---- Fixed figure numbers ----
figure_number_map = {"LG": (5,6), "BoE": (7,8), "Health": (9,10)}
sector_title_map  = {"LG":"Local Government", "BoE":"Board of Education", "Health":"HealthCare Sector"}

# ---- LG label mapping ----
LG_LABEL = {"LG-S1":"Scenario A","LG-S2":"Scenario B","LG-S3":"Scenario C","LG-S4":"Scenario D"}
LG_ORDER = ["LG-S1","LG-S2","LG-S3","LG-S4"]

# ---- Reader ----
def read_any_table(path: str) -> pd.DataFrame:
    ext = os.path.splitext(path)[1].lower()
    if ext in [".xlsx",".xlsm",".xls"]:
        return pd.read_excel(path, engine="openpyxl")
    if ext == ".csv":
        for enc in ["utf-8-sig","utf-8","cp932","shift_jis","latin1"]:
            try:
                return pd.read_csv(path, encoding=enc)
            except UnicodeDecodeError:
                continue
        return pd.read_csv(path, encoding="cp932", errors="replace")
    raise ValueError(f"Unsupported: {path}")

# ★重要：ファイル名からLG-S1..4を確実に決める
def infer_lg_scenario_id(fname: str) -> str | None:
    base = os.path.basename(fname).lower()
    m = re.search(r"scenario\s*([a-d])", base) or re.search(r"scenario([a-d])", base)
    if m:
        return {"a":"LG-S1","b":"LG-S2","c":"LG-S3","d":"LG-S4"}[m.group(1)]
    m = re.search(r"lg[\-_ ]?s([1-4])", base)
    if m:
        return f"LG-S{m.group(1)}"
    return None

def ensure_required(df: pd.DataFrame) -> pd.DataFrame:
    req = ["Scenario_ID","AttackTree_ID","Step_num","Ri"]
    miss = [c for c in req if c not in df.columns]
    if miss:
        raise ValueError(f"Missing columns: {miss}\nColumns={df.columns.tolist()}")

    df = df.copy()
    df["Scenario_ID"] = df["Scenario_ID"].astype(str)
    df["AttackTree_ID"] = pd.to_numeric(df["AttackTree_ID"], errors="coerce")
    df["Step_num"] = pd.to_numeric(df["Step_num"], errors="coerce")
    df["Ri"] = pd.to_numeric(df["Ri"], errors="coerce")
    df = df.dropna(subset=["AttackTree_ID","Step_num"]).copy()
    df["AttackTree_ID"] = df["AttackTree_ID"].astype(int)
    df["Step_num"] = df["Step_num"].astype(int)

    if "R_total" not in df.columns:
        last = (df.sort_values(["Scenario_ID","AttackTree_ID","Step_num"])
                  .groupby(["Scenario_ID","AttackTree_ID"], as_index=False)
                  .tail(1)[["Scenario_ID","AttackTree_ID","Ri"]]
                  .rename(columns={"Ri":"R_total"}))
        df = df.merge(last, on=["Scenario_ID","AttackTree_ID"], how="left")
    else:
        df["R_total"] = pd.to_numeric(df["R_total"], errors="coerce")
    return df

def compute_metrics(df: pd.DataFrame) -> pd.DataFrame:
    df = ensure_required(df)
    idx = df.groupby("Scenario_ID")["R_total"].idxmax()
    worst = df.loc[idx, ["Scenario_ID","AttackTree_ID"]].copy()

    rows = []
    for sc, tid in worst.itertuples(index=False):
        sub = df[(df["Scenario_ID"]==sc) & (df["AttackTree_ID"]==tid)].sort_values("Step_num")
        rows.append({
            "Scenario_ID": sc,
            "Rmax": float(np.nanmax(sub["Ri"].values)),
            "Rsum": float(np.nansum(sub["Ri"].values)),
            "E": float(np.nanmean(sub["Ri"].values)),
            "Ri_series": sub["Ri"].tolist()
        })
    return pd.DataFrame(rows)

def add_display(metrics: pd.DataFrame, sector: str) -> pd.DataFrame:
    m = metrics.copy()
    if sector == "LG":
        m["Display"] = m["Scenario_ID"].map(LG_LABEL).fillna(m["Scenario_ID"])
        m["_ord"] = m["Scenario_ID"].apply(lambda x: LG_ORDER.index(x) if x in LG_ORDER else 999)
        m = m.sort_values("_ord").drop(columns="_ord").reset_index(drop=True)
    else:
        m["Display"] = m["Scenario_ID"]
    return m

# ============================================================
# Upload
# ============================================================
sector_data = {}

# ---- LG: 4 files, one-by-one (必ず4回) ----
lg_dfs = []
print("\n=== LG: 4ファイルを『順番に』アップロード（A→B→C→D推奨）===")
for k in range(1,5):
    print(f"\n[LG] Upload {k}/4")
    up = files.upload()
    if len(up) != 1:
        raise RuntimeError("LGは1回につき1ファイルのみ（合計4回）アップしてください。")
    f = list(up.keys())[0]
    d = read_any_table(f)

    inferred = infer_lg_scenario_id(f)
    if inferred is None:
        raise RuntimeError(f"LGのシナリオ推定に失敗。ファイル名に scenarioA/B/C/D または LG-S1..4 を含めてください: {f}")

    # ★ここが今回の決定打：中身が全部LG-S1でも、ファイル名に従って強制上書き
    d = d.copy()
    d["Scenario_ID"] = inferred

    lg_dfs.append(d)

sector_data["LG"] = pd.concat(lg_dfs, ignore_index=True)

# ---- BoE (optional) ----
print("\n=== BoE: 1ファイルをアップロード（不要ならキャンセル）===")
boe_up = files.upload()
if len(boe_up) == 1:
    f = list(boe_up.keys())[0]
    d = read_any_table(f)
    if "Scenario_ID" not in d.columns or d["Scenario_ID"].astype(str).str.strip().eq("").all():
        d = d.copy()
        d["Scenario_ID"] = "BoE-S1"
    sector_data["BoE"] = d

# ---- Health (optional) ----
print("\n=== Health: 1ファイルをアップロード（不要ならキャンセル）===")
h_up = files.upload()
if len(h_up) == 1:
    f = list(h_up.keys())[0]
    d = read_any_table(f)
    if "Scenario_ID" not in d.columns or d["Scenario_ID"].astype(str).str.strip().eq("").all():
        d = d.copy()
        d["Scenario_ID"] = "Health-S1"
    sector_data["Health"] = d

# ---- Debug: ここで4つ出るのが必須 ----
print("\n=== DETECTED Scenario_IDs ===")
for sec, df in sector_data.items():
    print(sec, "->", sorted(df["Scenario_ID"].astype(str).unique().tolist()))

# ============================================================
# Plot (ONE figure per sector)
# ============================================================
outputs = []

for sector in ["LG","BoE","Health"]:
    if sector not in sector_data:
        continue

    metrics = compute_metrics(sector_data[sector])
    metrics = add_display(metrics, sector)

    figA, figB = figure_number_map[sector]
    sec_name = sector_title_map[sector]

    # Fig A (bars)
    plt.figure(figsize=(7,5))
    x = np.arange(len(metrics))
    width = 0.25
    plt.bar(x-width, metrics["Rmax"], width, label=r"$R_{max}$")
    plt.bar(x,       metrics["Rsum"], width, label=r"$R_{sum}$")
    plt.bar(x+width, metrics["E"],    width, label=r"$E$")
    plt.xticks(x, metrics["Display"], rotation=30, ha="right")
    plt.ylabel("Score")
    plt.title(f"Fig. {figA}. $R_{{max}}$, $R_{{sum}}$, and $E$ (Worst-case attack tree, {sec_name})")
    plt.legend(frameon=False)
    plt.tight_layout()
    outA = f"Fig{figA}_{sector}.png"
    plt.savefig(outA, dpi=DPI, bbox_inches="tight")
    plt.close()
    outputs.append(outA)

    # Fig B (lines)
    plt.figure(figsize=(7,5))
    for _, row in metrics.iterrows():
        steps = list(range(1, len(row["Ri_series"])+1))
        plt.plot(steps, row["Ri_series"], marker="o", linewidth=2, label=row["Display"])
    plt.xlabel("Step $i$")
    plt.ylabel("Residual risk $R_i$")
    plt.title(f"Fig. {figB}. Stepwise residual risk $R_i$ (Worst-case attack tree, {sec_name})")
    plt.legend(frameon=False)
    plt.tight_layout()
    outB = f"Fig{figB}_{sector}.png"
    plt.savefig(outB, dpi=DPI, bbox_inches="tight")
    plt.close()
    outputs.append(outB)

print("\n=== Auto download ===")
for f in outputs:
    files.download(f)

print("✔ 完了：LGのFig5/6はScenario A〜Dが同じグラフ内に4つ出ます。")



=== LG: 4ファイルを『順番に』アップロード（A→B→C→D推奨）===

[LG] Upload 1/4


Saving AASG_SPRV_FREQUENCY_LG_LONG_scenarioA_with_formulas.xlsx to AASG_SPRV_FREQUENCY_LG_LONG_scenarioA_with_formulas (5).xlsx

[LG] Upload 2/4


Saving AASG_SPRV_FREQUENCY_LG_LONG_scenarioB_with_formulas.xlsx to AASG_SPRV_FREQUENCY_LG_LONG_scenarioB_with_formulas (5).xlsx

[LG] Upload 3/4


Saving AASG_SPRV_FREQUENCY_LG_LONG_scenarioC_with_formulas.xlsx to AASG_SPRV_FREQUENCY_LG_LONG_scenarioC_with_formulas (5).xlsx

[LG] Upload 4/4


Saving AASG_SPRV_FREQUENCY_LG_LONG_scenarioD_with_formulas.xlsx to AASG_SPRV_FREQUENCY_LG_LONG_scenarioD_with_formulas (5).xlsx

=== BoE: 1ファイルをアップロード（不要ならキャンセル）===


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✔ 完了：LGのFig5/6はScenario A〜Dが同じグラフ内に4つ出ます。
